In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [5]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.55, 1.6, 1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.5, 0.6, 0.7, 0.9)]
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.232588                    1  0.006292
0.000750        2  0.261665                    2  0.044196
0.001052        3  0.381656                    3  0.138900
0.001476        4  0.512177                    4  0.246390
0.002070        5  0.648170                    5  0.365584
0.002904        6  0.785489                    6  0.495104
0.004074        7  0.917783                    7  0.636461
0.005714        8  1.037483                    8  0.786961
0.008015        9  1.136182                    9  0.938234
0.011243       10  1.205306                   10  1.081276
0.015771       11  1.238317                   11  1.204262
0.022122       12  1.233512                   12  1.292527
0.031031       13  1.197010                   13  1.329189
0.043528       14  1.106109                   14  1.268594
0.061057       15  0.951866                   15  1.086910
0.085645       16  0.794435                   16  0.850796
0.120136       17  0.672804                   17  0.640870
0.168516       18  0.587409                   18  0.504073
0.236378       19  0.538689                   19  0.453883
0.331549       20  0.530741                   20  0.472236
0.465100       21  0.583073                   21  0.556316
0.652400       22  0.542708                   22  0.543188
0.915100       23  0.397238                   23  0.411969
1.283650       24  0.290295                   24  0.303006
1.800600       25  0.221878                   25  0.229946
2.525700       26  0.169522                   26  0.177111
3.542800       27  0.125489                   27  0.137019
4.969550       28  0.095703                   28  0.108557
6.970850       29  0.081750                   29  0.092898
9.778100       30  0.071159                   30  0.077528
13.715850      31  0.058615                   31  0.059401
19.239350      32  0.045015                   32  0.039207
26.987250      33  0.039193                   33  0.030072
37.855300      34  0.041335                   34  0.035111
53.100050      35  0.042569                   35  0.042011
73.887500      36  0.043279                   36  0.046858
97.662500      37  0.043368                   37  0.047621
121.437500     38  0.040081                   38  0.043270
145.212500     39  0.035064                   39  0.036720
168.987500     40  0.030671                   40  0.030990
192.762500     41  0.026218                   41  0.025480
216.537500     42  0.021438                   42  0.019859
240.312500     43  0.016018                   43  0.013473
264.087500     44  0.009148                   44  0.005437
287.862500     45 -0.000340                   45 -0.005159
311.637500     46 -0.003842                   46 -0.009093
335.412500     47  0.004204                   47 -0.000602
359.187500     48  0.014776                   48  0.010900
382.962500     49  0.023278                   49  0.020161
406.737500     50  0.031009                   50  0.028387
430.512500     51  0.038396                   51  0.036195
454.287500     52  0.045616                   52  0.043857
478.062500     53  0.052741                   53  0.051716
501.837500     54  0.059764                   54  0.059538
525.612500     55  0.066625                   55  0.067142
549.387500     56  0.073188                   56  0.074630
573.162500     57  0.079294                   57  0.081746
596.937500     58  0.084754                   58  0.088208
620.712500     59  0.089392                   59  0.093640
644.487500     60  0.093096                   60  0.098047
668.262500     61  0.095844                   61  0.101171
692.037500     62  0.097750                   62  0.103372
715.812500     63  0.099014                   63  0.104857
739.587500     64  0.099933                   64  0.105933
763.362500     65  0.100863                   65  0.106954
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -15.591444   0.000000 -15.591444 -15.423142  2.302671e-09   
0.000624    2     -15.591497   0.000071 -15.591427 -15.423192  5.033838e-05   
0.000876    3     -15.591519   0.000100 -15.591419 -15.423212  7.166695e-05   
0.001229    4     -15.591548   0.000145 -15.591403 -15.423239  1.049498e-04   
0.001723    5     -15.591588   0.000215 -15.591373 -15.423277  1.567759e-04   
0.002417    6     -15.591641   0.000322 -15.591319 -15.423327  2.371264e-04   
0.003391    7     -15.591712   0.000483 -15.591229 -15.423394  3.610809e-04   
0.004757    8     -15.591804   0.000724 -15.591080 -15.423482  5.521784e-04   
0.006672    9     -15.591923   0.001078 -15.590845 -15.423597  8.452988e-04   
0.009359    10    -15.592073   0.001589 -15.590483 -15.423742  1.289794e-03   
0.013128    11    -15.592258   0.002313 -15.589945 -15.423923  1.953555e-03   
0.018415    12    -15.592481   0.003311 -15.589170 -15.424140  2.924791e-03   
0.025830    13    -15.592736   0.004649 -15.588086 -15.424388  4.307560e-03   
0.036232    14    -15.593003   0.006392 -15.586611 -15.424645  6.203226e-03   
0.050823    15    -15.593258   0.008559 -15.584699 -15.424890  8.641091e-03   
0.071291    16    -15.593503   0.011111 -15.582392 -15.425121  1.150707e-02   
0.100000    17    -15.593732   0.014042 -15.579690 -15.425328  1.460792e-02   
0.140271    18    -15.593901   0.017421 -15.576480 -15.425464  1.780181e-02   
0.196760    19    -15.593925   0.021375 -15.572550 -15.425445  2.115542e-02   
0.275997    20    -15.593638   0.026145 -15.567493 -15.425115  2.508663e-02   
0.387100    21    -15.592669   0.032161 -15.560508 -15.424132  3.031966e-02   
0.543100    22    -15.589778   0.040044 -15.549734 -15.421225  3.769402e-02   
0.761700    23    -15.584534   0.048854 -15.535680 -15.415915  4.645125e-02   
1.068500    24    -15.578411   0.057167 -15.521244 -15.409635  5.514479e-02   
1.498800    25    -15.572092   0.065646 -15.506447 -15.403052  6.400830e-02   
2.102400    26    -15.565602   0.075019 -15.490582 -15.396166  7.356516e-02   
2.949000    27    -15.559092   0.085510 -15.473582 -15.389135  8.429774e-02   
4.136600    28    -15.553225   0.097297 -15.455928 -15.382566  9.700638e-02   
5.802500    29    -15.548535   0.111493 -15.437042 -15.376971  1.128353e-01   
8.139200    30    -15.544597   0.130183 -15.414414 -15.371833  1.334142e-01   
11.417000   31    -15.541348   0.154564 -15.386785 -15.367064  1.587506e-01   
16.014700   32    -15.540057   0.185196 -15.354861 -15.364063  1.881049e-01   
22.464000   33    -15.544210   0.223739 -15.320471 -15.366938  2.209363e-01   
31.510500   34    -15.556127   0.277655 -15.278472 -15.378338  2.645652e-01   
44.200100   35    -15.574734   0.358396 -15.216337 -15.396446  3.354576e-01   
62.000000   36    -15.601323   0.474744 -15.126579 -15.422808  4.504092e-01   
85.775000   37    -15.636476   0.631783 -15.004693 -15.458783  6.183633e-01   
109.550000  38    -15.671085   0.788531 -14.882554 -15.495040  7.887518e-01   
133.325000  39    -15.708034   0.938360 -14.769674 -15.534251  9.498357e-01   
157.100000  40    -15.750898   1.079975 -14.670923 -15.579716  1.098726e+00   
180.875000  41    -15.800417   1.215874 -14.584543 -15.632112  1.238407e+00   
204.650000  42    -15.857472   1.346766 -14.510707 -15.692426  1.370488e+00   
228.425000  43    -15.923332   1.473002 -14.450330 -15.762800  1.496797e+00   
252.200000  44    -16.000052   1.594832 -14.405219 -15.845843  1.617789e+00   
275.975000  45    -16.092002   1.712547 -14.379454 -15.946746  1.734007e+00   
299.750000  46    -16.209345   1.828932 -14.380413 -16.075890  1.848621e+00   
323.525000  47    -16.353265   1.962032 -14.391234 -16.233067  1.980187e+00   
347.300000  48    -16.510639   2.131246 -14.379394 -16.403683  2.149109e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')